## Conceptos importantes de la preparación de datos para ML

¿Cuales son los primeros pasos que tenemos que dar a la hora de hacer un análisis de datos? ¿Y si además tenemos que armar un modelo de machine learning?
Existen múltiples caminos que podemos seguir para cumplir con estos objetivos. Podríamos parte de la siguiente categorización:

* Data collection (Data Engineering)
* Análisis exploratorio de los datos (EDA)
* Preparación de los datos (Data Wrangling)
* Selección de variables (Data Wrangling)
* Data Integration (Data Engineering)
* Machine learning - Deep Learning (Data Scientist - MLops)
* Evaluación del modelo (Data Scientist - MLops)

Parece intuitivo el ciclo de vida del dato (pipeline de los datos). No habiendo mucho más que explicar, en esta parte estaremos repasando los principales conceptos que hacen a un análisis exploratorio de los datos y a la preparación de los datos para luego confeccionar un modelo de ML como también para tomar decisiones.

##### Breve reseña de los datos:

En este dataset, cada fila representa una persona que saco un crédito en un banco. De acuerdo con sus atributos cada persona fue clasificada como un buen o un mal pagador. Este dataset es una adaptación de un dataset original cuyo creador es Hans Hofmann. Algunas de las variables de este dataset son:

* Age (numeric)
* Sex (text: male, female)
* Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
* Housing (text: own, rent, or free)
* Saving accounts (text - little, moderate, quite rich, rich)
* Checking account (numeric, in DM - Deutsch Mark)
* Credit amount (numeric, in DM)
* Duration (numeric, in month)
* Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)
* Risk (Value target - Good or Bad Risk)

Fuentes:

* https://www.kaggle.com/datasets/kabure/german-credit-data-with-risk
* https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data

In [1]:
# Importamos las librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Importamos los datos
df = pd.read_csv("../data/german_credit_data.xls", index_col=0)
df.head(10)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
5,35,male,1,free,NaN,NaN,9055,36,education,good
6,53,male,2,own,quite rich,NaN,2835,24,furniture/equipment,good
7,35,male,3,rent,little,moderate,6948,36,car,good
8,61,male,1,own,rich,NaN,3059,12,radio/TV,good
9,28,male,3,own,little,moderate,5234,30,car,bad


In [12]:
# Corroboramos el tipo de objeto que levantamos (no es necesario. Fines didácticos)
type(df)

pandas.core.frame.DataFrame

##### Conceptos importantes y diferencias entre DataFrames y Series:

Podemos ver que siempre que creemos un dataFrame o una Serie, vamos a ver un indice (index). Vemos que cuando creamos uno de estos objetos y queremos ver que tipo de datos tenemos vemos un "dtype". El dtype es el tipo de datos de cada uno de los objetos dentro de la serie o del dataframe. Los objetos que usualmente se llaman strings acá los llamamos "object".

En particular, una Serie es una lista de datos indexada que mantiene el tipo de datos. Por otro lado, los dataFrames serían una especie de agrupado de Series en función de un mismo indice. A diferencia de lo que vemos en Excel o en otros programas. La diferencia con una matriz convencional es que este tipo de objeto nos permite tener dentro distintos tipos de datos.

#### Análisis exploratorio de los datos (EDA)

In [9]:
# Lo primero que tenemos que hacer si es que aún lo desconocemos es saber la cantidad de filas 
# y columnas del dataset
df.shape

(1000, 10)

In [10]:
# Lo segundo que tenemos que ver es el tipo de datos
df.dtypes

Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
Risk                object
dtype: object

In [11]:
# Generalmente, si bien la anterior forma sirve, utilizamos el método info dado que también nos aporta 
# información sobre los nulos del dataset que estamos analizando
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 85.9+ KB


In [13]:
# Lo que podemos ver con esta función es que además parece que tenemos nulos en dos variables.
# Otra forma de chequear esto es:
df.isnull().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [14]:
# Por último, tenemos el método describe que nos muestra estadísticos comunes para las variables numéricas
df.describe()

,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,3271.258000,20.903000
std,11.375469,0.653614,2822.736876,12.058814
min,19.000000,0.000000,250.000000,4.000000
25%,27.000000,2.000000,1365.500000,12.000000
50%,33.000000,2.000000,2319.500000,18.000000
75%,42.000000,2.000000,3972.250000,24.000000
max,75.000000,3.000000,18424.000000,72.000000


In [19]:
# Ahora procedemos con un análisis de variables categóricas
cat_cols = [col for col in df.columns if df[col].dtypes == 'O']
print("Variables categóricas: ", cat_cols, "\n")

for col in cat_cols:
    print(df[col].value_counts(), "\n\n")

Variables categóricas:  ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk'] 

Sex
male      690
female    310
Name: count, dtype: int64 


Housing
own     713
rent    179
free    108
Name: count, dtype: int64 


Saving accounts
little        603
moderate      103
quite rich     63
rich           48
Name: count, dtype: int64 


Checking account
little      274
moderate    269
rich         63
Name: count, dtype: int64 


Purpose
car                    337
radio/TV               280
furniture/equipment    181
business                97
education               59
repairs                 22
domestic appliances     12
vacation/others         12
Name: count, dtype: int64 


Risk
good    700
bad     300
Name: count, dtype: int64 




In [ ]:
# Posteriormente tenemos que seguir con el análisis univariado de las variables numéricas (estadísticas)

In [ ]:
# En particular, deberíamos hacer un análisis aún más puntilloso de la variable dependiente del modelo (distribución)

In [ ]:
# Análisis bivariado y multivariado

#### Data Preprocessing

In [20]:
# Manejo de tipos de datos

In [21]:
# Maniejo  de nulos

In [ ]:
# Manejo de outliers

In [22]:
# Estandarizacion

In [23]:
# Discretización de variables numéricas

In [24]:
# Variables de alta cardinalidad

In [25]:
# Anáisis de correlaciones

In [26]:
# Selección de vairbales